In [1]:
DF_PATH       = "../data/processed/2_cleaned_data.pkl"

ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageWorkedWith',    'DatabaseWorkedWith',    'WebframeWorkedWith',    'MiscTechWorkedWith']

EXPERIMENT_NAME = "stackoverflow_single_model"
LOG_PATH = "../models/temp/random_forest/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt

## Read data

In [3]:
# Read Data 
df = pd.read_pickle(DF_PATH)

## Functions

In [4]:
def calculate_quality(ground_truth, prediction, metric_function, sort_values=False):
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    
    return quality_scores


## Balance classes

In [5]:
# Check the total samples of roles
roles_df = df["DevType"].copy()
role_sum = df["DevType"].sum(axis=0)
role_sum

Academic researcher                               581
Data or business analyst                          669
Data scientist or machine learning specialist     799
Database administrator                            296
DevOps specialist                                 677
Developer, QA or test                             493
Developer, back-end                              5503
Developer, desktop or enterprise applications    1671
Developer, embedded applications or devices       795
Developer, front-end                             2890
Developer, full-stack                            5578
Developer, game or graphics                       342
Developer, mobile                                1859
Engineer, data                                    483
Scientist                                         292
System administrator                              440
dtype: int64

In [6]:
# Resample roles
samples_per_class = 500 
resampled_roles = []

for role_col in roles_df.columns:
    sub_df = roles_df.loc[roles_df[role_col] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # Upsample(bootstrap sampling with replacement)
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=0)
    else:
        # Downsample (selection from population)
        sub_df = sub_df.sample(samples_per_class, random_state=0) 
    
    resampled_roles.append(sub_df)

In [7]:
# Construct dfs
roles_df  = pd.concat(resampled_roles)
df = df.loc[roles_df.index]

In [8]:
#it will be greater than 500 because the row contains two answers
roles_df.sum(axis=0) 

Academic researcher                               797
Data or business analyst                          767
Data scientist or machine learning specialist     871
Database administrator                            588
DevOps specialist                                 705
Developer, QA or test                             590
Developer, back-end                              1770
Developer, desktop or enterprise applications     892
Developer, embedded applications or devices       666
Developer, front-end                              893
Developer, full-stack                            1431
Developer, game or graphics                       560
Developer, mobile                                 800
Engineer, data                                    631
Scientist                                         642
System administrator                              647
dtype: int64

## Prep

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop("DevType", axis=1), 
                                                    df["DevType"], 
                                                    random_state=0)

C:\Users\Omnia Hosny\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [10]:
X_train.shape

(6000, 91)

## Train models

In [11]:
# Initialize client and experiment
client = MlflowClient()
mlflow.set_experiment(EXPERIMENT_NAME)
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

## 1. Vanilla Forest

In [12]:
rf_clf = make_pipeline(StandardScaler(),
                       RandomForestClassifier(random_state=0))

rf_clf.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=0))])

In [13]:
# Evaluate on train set
predictions =  pd.DataFrame(rf_clf.predict(X_train), columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

In [14]:
# Evaluate on test set
predictions =  pd.DataFrame(rf_clf.predict(X_test), columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)
mean_test_scores = test_scores.mean()

In [15]:
print(test_scores.mean())
test_scores.sort_values("precision_score")

accuracy_score     94.196875
precision_score    83.858750
recall_score       57.422500
f1_score           67.671250
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
"Developer, full-stack",86.50,70.42,42.02,52.63
"Developer, desktop or enterprise applications",90.95,70.79,28.90,41.04
"Developer, embedded applications or devices",94.30,73.15,48.17,58.09
"Developer, front-end",93.30,74.00,53.88,62.36
"Developer, back-end",85.20,79.07,45.74,57.95
Data scientist or machine learning specialist,95.30,81.82,69.90,75.39
Data or business analyst,95.10,83.33,54.60,65.97
Academic researcher,94.05,83.85,52.66,64.69
Database administrator,97.10,84.00,73.43,78.36
Scientist,97.15,88.98,70.47,78.65


## Log

In [16]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [17]:
# Model
model = {"model_description": "Random Forest: with non linearity",
         "model_details": str(rf_clf),
         "model_object": rf_clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [18]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [19]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, 
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)  
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 
    

## 2. Random Forest with Non-linearity

In [20]:
rf_clf = make_pipeline(StandardScaler(), 
                       FeatureUnion([('linear_pca', PCA(n_components=40)), 
                                      ('kernel_pca', KernelPCA(n_components=40, 
                                                               kernel='rbf'))]),
                       RandomForestClassifier(random_state=0))

rf_clf.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('featureunion',
                 FeatureUnion(transformer_list=[('linear_pca',
                                                 PCA(n_components=40)),
                                                ('kernel_pca',
                                                 KernelPCA(kernel='rbf',
                                                           n_components=40))])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=0))])

In [21]:
# Evaluate on train set
predictions =  pd.DataFrame(rf_clf.predict(X_train), columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

In [22]:
# Evaluate on test set
predictions =  pd.DataFrame(rf_clf.predict(X_test), columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)
mean_test_scores = test_scores.mean()

In [23]:
print(mean_test_scores)
test_scores.sort_values("f1_score")

accuracy_score     94.193750
precision_score    87.710000
recall_score       54.523125
f1_score           66.478125
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
"Developer, desktop or enterprise applications",91.70,90.62,26.61,41.13
"Developer, full-stack",86.50,73.26,38.38,50.37
"Developer, embedded applications or devices",94.20,76.67,42.07,54.33
"Developer, back-end",84.15,74.52,43.95,55.29
"Developer, front-end",93.60,81.97,48.54,60.98
Academic researcher,94.00,91.43,46.38,61.54
DevOps specialist,95.00,92.31,47.46,62.69
Data or business analyst,95.25,87.62,52.87,65.95
"Developer, QA or test",96.20,95.74,55.56,70.31
Data scientist or machine learning specialist,95.10,86.00,62.62,72.47


In [24]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [25]:
# Model
model = {"model_description": "Random Forest: with PCAs + RBF kernel",
         "model_details": str(rf_clf),
         "model_object": rf_clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [26]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [27]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, 
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)  
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 

## 2. Random Forest with PCAs & Hyper parameter tuning

In [28]:
hpt_rf_clf = make_pipeline(StandardScaler(), 
                           FeatureUnion([('linear_pca', PCA()), 
                                         ('kernel_pca', KernelPCA(kernel='rbf'))]),
                           RandomForestClassifier(random_state=0, ))

In [ ]:
# list(hpt_rf_clf.get_params().keys())
tuned_parameters = [{'featureunion__linear_pca__n_components': [5, 10, 20, 40, 60], 
                     'featureunion__kernel_pca__n_components': [5, 10, 20, 40, 60],
                     'randomforestclassifier__n_estimators':   [100, 500, 1000]}]
hpt_rf_clf = GridSearchCV(hpt_rf_clf, 
                          tuned_parameters, 
                          n_jobs=-1)

hpt_rf_clf.fit(X_train, Y_train)

In [ ]:
hpt_rf_clf.best_params_

In [ ]:
# Evaluate on test set
predictions =  pd.DataFrame(hpt_rf_clf.predict(X_test), columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)
print(test_scores.mean())
test_scores.sort_values("f1_score")

## Log

In [ ]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [ ]:
# Model
model = {"model_description": "Random Forest: with PCAs + RBF kernel + Hyperparamter tuning",
         "model_details": str(hpt_rf_clf),
         "model_object": hpt_rf_clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [ ]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [ ]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, 
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)  
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 
    